### Experimenting with PCA and Mean-Reversion Strategy

In [24]:
# Loading libraries.

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib as mpl

import seaborn as sns

import time

import datetime as dt
import re

import pandas_datareader.data as web
from pandas_datareader import data as pdr

import warnings
warnings.filterwarnings("ignore")

from sklearn.preprocessing import StandardScaler

from sklearn.decomposition import PCA
from sklearn.decomposition import KernelPCA

from sklearn.decomposition import TruncatedSVD

from numpy.linalg import inv, eig, svd

from statsmodels.tsa.stattools import coint
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller

from itertools import combinations


%matplotlib inline

In [25]:
#Loading Dataset
df = pd.read_csv(r'C:/Users/glori/Downloads/dow_2000_2019.csv', index_col=0)
df

,MMM,AXP,AAPL,BA,CAT,CVX,CSCO,KO,DIS,DWDP,...,NKE,PFE,PG,TRV,UTX,UNH,VZ,V,WMT,WBA
Date,,,,,,,,,,,,,,,,,,,,,
03-01-00,29.847043,35.476634,3.530576,26.650218,14.560887,21.582046,43.003876,16.983583,23.522220,NaN,...,4.701180,16.746856,32.227726,20.158885,21.319030,5.841355,22.564221,NaN,47.337599,21.713237
04-01-00,28.661131,34.134275,3.232839,26.610431,14.372251,21.582046,40.577200,17.040950,24.899860,NaN,...,4.445214,16.121738,31.596399,19.890099,20.445803,5.766368,21.833915,NaN,45.566248,20.907354
05-01-00,30.122175,33.959430,3.280149,28.473758,14.914205,22.049145,40.895453,17.228147,25.781550,NaN,...,4.702157,16.415912,31.325831,20.085579,20.254784,5.753327,22.564221,NaN,44.503437,21.097421
06-01-00,31.877325,33.959430,2.996290,28.553331,15.459153,22.903343,39.781569,17.210031,24.899860,NaN,...,4.677733,16.972739,32.438168,20.122232,20.998392,5.964159,22.449405,NaN,45.126952,20.527220
07-01-00,32.509812,34.433913,3.138219,29.382213,15.962182,23.305926,42.128682,18.342270,24.506249,NaN,...,4.677733,18.123166,35.023602,20.922479,21.830687,6.662948,22.282692,NaN,48.535033,21.051805
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25-01-19,195.900000,100.770000,157.760000,364.200000,136.860000,113.220000,46.130000,47.370000,111.090000,57.76,...,80.610000,40.298248,93.600000,124.980000,115.810000,268.050000,56.400000,138.67,96.940000,71.890000
28-01-19,193.200000,100.380000,156.300000,362.970000,124.370000,112.170000,45.750000,47.170000,110.810000,58.13,...,80.320000,39.197582,93.520000,125.030000,115.080000,266.770000,55.070000,135.99,97.060000,71.580000
29-01-19,196.950000,100.960000,154.680000,364.910000,126.530000,111.830000,45.960000,47.400000,110.900000,58.52,...,80.220000,40.427155,93.540000,124.820000,117.840000,267.340000,53.280000,135.00,96.710000,71.500000


In [26]:
#Cleaning Data
missing_values = df.isnull().mean()
##2 missing values in dataset (DWDP(0.925208) and V(0.429792))

drop_list = sorted(list(missing_values[missing_values > 0.04].index))

df = df.drop(labels=drop_list, axis=1)
#df


In [27]:
#Calculating Linear Returns and Scaling it for PCA
df_linear_returns = df.pct_change(1)
df_linear_returns =\
(
    df_linear_returns[df_linear_returns 
                        .apply(lambda x:(x - x.mean()
                                        ).abs() < (3 * x.std()
                                                  )
                              )
                        .all(1)
    ]
)

scaler = StandardScaler().fit(df_linear_returns)

df_scaler = pd.DataFrame(scaler.fit_transform(df_linear_returns), columns=df_linear_returns.columns, index=df_linear_returns.index)

### Obtaining an Eigen Portfolio

In [28]:
#Spiltting the Data
prop =\
    int(len(df_scaler) * 0.80)

X_Train = df_scaler[    : prop] 
X_Test  = df_scaler[prop:     ] 

X_Train_Raw = df_linear_returns[    :prop]
X_Test_Raw  = df_linear_returns[prop:    ]

In [29]:
#Running PCA
pca = PCA()

PrincipalComponent = pca.fit(X_Train)

In [30]:
#Examining the PCs' explained variance and PCs' scores
pd.Series(np.cumsum(pca.explained_variance_ratio_))
#scores = pd.DataFrame(pca.transform(X_Train),index=X_Train.index)

0     0.370367
1     0.427534
2     0.471104
3     0.510838
4     0.546057
5     0.577487
6     0.606540
7     0.634420
8     0.661910
9     0.687143
10    0.711658
11    0.735528
12    0.757933
13    0.779061
14    0.799345
15    0.819558
16    0.838884
17    0.858008
18    0.876060
19    0.893804
20    0.910545
21    0.926178
22    0.941469
23    0.955956
24    0.970076
25    0.982451
26    0.993257
27    1.000000
dtype: float64

In [31]:
#Choosing Eigen Portfolio based on Sharpe Ratio
def calculate_sharpe_ratio(ts_returns, periods_per_year = 252):
    n_years = ts_returns.shape[0] / periods_per_year
    annualized_return = np.power(np.prod(1 + ts_returns), (1 / n_years))-1
    annualized_vol = ts_returns.std() * np.sqrt(periods_per_year)
    annualized_sharpe = annualized_return / annualized_vol
    return annualized_return, annualized_vol, annualized_sharpe


def eigen_portfolios():
    
    n_portfolios = len(pca.components_)
    
    annualized_ret = np.array([0.] * n_portfolios)

    sharpe_metric = np.array([0.] * n_portfolios)

    annualized_vol = np.array([0.] * n_portfolios)
    
    highest_sharpe = 0
    
    stock_tickers =\
    (df_scaler
     .columns 
     .values)

    n_tickers = len(stock_tickers)
    
    PCs = pca.components_
    for i in range(n_portfolios):
        pc_w = PCs[i] / sum(PCs[i])
        eigen_prtfi =\
            (pd.DataFrame(data = {"weights": pc_w.squeeze() * 100},
                           index = stock_tickers))
        
        eigen_prti_returns =\
            (np.dot(X_Train_Raw,
                     pc_w))

        eigen_prti_returns =\
            (pd.Series(eigen_prti_returns.squeeze(),
                        index = X_Train_Raw.index))

        er, vol, sharpe = calculate_sharpe_ratio(eigen_prti_returns)
        annualized_ret[i] = er
        annualized_vol[i] = vol
        sharpe_metric[i] = sharpe
        
        sharpe_metric = np.nan_to_num(sharpe_metric)

    highest_sharpe = np.argmax(sharpe_metric)
    
    results = pd.DataFrame(data = {"Return": annualized_ret,
                               "Vol": annualized_vol,
                               "Sharpe": sharpe_metric})

    results.dropna(inplace = True)

    results.sort_values(by = ["Sharpe"],
                        ascending = False,
                        inplace = True)

    print(results.head())

In [32]:
eigen_portfolios()

      Return        Vol    Sharpe
0   0.115981   0.134411  0.862886
10  0.494180   0.924699  0.534422
5   0.297760   1.136205  0.262065
18 -1.000000  19.450861 -0.051412
7  -0.061040   0.796040 -0.076679


In [33]:
pca.components_[:,0]

array([ 0.22774626, -0.02389851,  0.13300255,  0.02105331, -0.12887666,
        0.05273942,  0.02281497,  0.1978921 , -0.0668771 , -0.02849654,
       -0.03398398, -0.06377015, -0.11940054, -0.02790107,  0.18063321,
        0.17605172,  0.1521066 , -0.32202404, -0.20236572, -0.02332706,
        0.04024685,  0.13726868, -0.08586135,  0.33156191, -0.54863893,
       -0.4164777 ,  0.02499486, -0.00291259])

In [34]:
#Obtaining the weights of Portfolio 
Portfolio_weights = pd.DataFrame(pca.components_[:], columns = df.columns)
Portfolio_weights = Portfolio_weights.div(Portfolio_weights.sum(1),axis = 0)
Portfolio_weights.iloc[0,:]

MMM     0.043399
AXP     0.043523
AAPL    0.029155
BA      0.036177
CAT     0.038487
CVX     0.033941
CSCO    0.037154
KO      0.032085
DIS     0.039660
XOM     0.036810
GS      0.039793
HD      0.038581
IBM     0.037977
INTC    0.037205
JNJ     0.031800
JPM     0.043456
MCD     0.028888
MRK     0.032408
MSFT    0.036928
NKE     0.032630
PFE     0.036196
PG      0.032003
TRV     0.037657
UTX     0.043170
UNH     0.026311
VZ      0.031815
WMT     0.032181
WBA     0.030611
Name: 0, dtype: float64

### Experimenting with Mean Reversion Strategy

In [288]:
#Selecting the 2 stocks with the highest and lowest weighting (AXP, UNH)
log_price = np.log(df[["AXP", "UNH"]])
score, pvalue, _ = coint(log_price.iloc[:,0], log_price.iloc[:,1])
pvalue

0.0326747243307142

In [289]:
#Calculating Hedge Ratio
AXP = log_price.iloc[:,0]
UNH = log_price.iloc[:,1]
model = sm.OLS(AXP, sm.add_constant(UNH)).fit()
hedge_ratio = model.params[1]
hedge_ratio

0.44934724039602403

In [292]:
#Calculating the Spread and Testing the Stationarity
spread = AXP - hedge_ratio * UNH

adfuller(spread, maxlag=0)[1]


0.004392471244286607

In [ ]:
#Mean Reversion Strategy 
entry_threshold = 1 * spread.mean()
signals = np.where(spread > entry_threshold, -1, np.nan) 
signals = np.where(spread < entry_threshold, 1, signals)  
signals = pd.Series(signals, index=spread.index).fillna(0)

#Calculating daily trading returns
returns = log_price.pct_change().dropna()
returns['strategy'] = signals.shift(1) * UNH - signals.shift(1) * AXP * hedge_ratio

#Calculating trading costs, assuming no trading fee
trade_signal = signals.diff().fillna(0)  
transaction_cost = 0.00   
cost = (trade_signal.abs() * transaction_cost).shift(1) * (UNH.abs() + AXP.abs() * hedge_ratio)

# Calculate net strategy returns after deducting trading costs
returns['net_strategy'] = returns['strategy'] - cost

In [291]:
def pnlPerformance(pnl, label):
    sharpe = pnl.mean() / np.std(pnl)
    sharpe = sharpe * np.sqrt(252)
    print("")
    print("PERFORMANCE STATISTICS FOR " + label)
    print("Daily annualized sharpe: " + str(sharpe))
    
# Evaluate strategy performance
pnlPerformance(returns['net_strategy'],f'Pairs Trading Strategy')


PERFORMANCE STATISTICS FOR Pairs Trading Strategy
Daily annualized sharpe: 1.136244904135691
